In [1]:
import os
import sys
import Wind_KAN_
import importlib
import time
import matplotlib.pyplot as plt

In [2]:
from Wind_KAN_ import *
from kan import *
torch.set_default_dtype(torch.float64)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
importlib.reload(Wind_KAN_)

# Loop through methods in the new class and update them on the existing instance
for method_name in dir(Wind_KAN_.Wind_KAN):
    if callable(getattr(Wind_KAN_.Wind_KAN, method_name)) and not method_name.startswith("__"):
        new_method = getattr(Wind_KAN_.Wind_KAN, method_name)
        setattr(instance_of_class, method_name, new_method.__get__(instance_of_class, instance_of_class.__class__))